In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('train.csv')
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data = raw_data.copy()
data['Sex'] = pd.get_dummies(data['Sex'], drop_first = True)
data= data.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis = 1)

In [4]:
data[["Embarked_Q", "Embarked_S"]] = pd.get_dummies(data['Embarked'], drop_first = True)
data = data.drop('Embarked', axis = 1)

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,1
1,1,1,0,38.0,1,0,71.2833,0,0
2,1,3,0,26.0,0,0,7.9250,0,1
3,1,1,0,35.0,1,0,53.1000,0,1
4,0,3,1,35.0,0,0,8.0500,0,1


In [6]:
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [7]:
data['Age'].isna().sum()

0

In [8]:
data[['Pclass_2', 'Pclass_3']] = pd.get_dummies(data['Pclass'], drop_first = True)
data = data.drop(['Pclass'], axis = 1)

In [9]:
data.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,1,22.0,1,0,7.2500,0,1,0,1
1,1,0,38.0,1,0,71.2833,0,0,0,0
2,1,0,26.0,0,0,7.9250,0,1,0,1
3,1,0,35.0,1,0,53.1000,0,1,0,0
4,0,1,35.0,0,0,8.0500,0,1,0,1


## Standardize data

In [10]:
from sklearn import preprocessing
data[['Age', 'Fare']] = preprocessing.scale(data[['Age', 'Fare']])
data

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,1,-0.592481,1,0,-0.502445,0,1,0,1
1,1,0,0.638789,1,0,0.786845,0,0,0,0
2,1,0,-0.284663,0,0,-0.488854,0,1,0,1
3,1,0,0.407926,1,0,0.420730,0,1,0,0
4,0,1,0.407926,0,0,-0.486337,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,1,-0.207709,0,0,-0.386671,0,1,1,0
887,1,0,-0.823344,0,0,-0.044381,0,1,0,0
888,0,0,0.000000,1,2,-0.176263,0,1,0,1
889,1,1,-0.284663,0,0,-0.044381,0,0,0,0


## Balance the dataset

In [11]:
data_dead_drop = len(data['Survived']) - data['Survived'].sum() * 2
for i in range(data.shape[0]):
    if data['Survived'][i] == 0:
        data = data.drop(i, axis = 0)
        data_dead_drop -= 1
    if data_dead_drop == 0:
        break
data.shape

(684, 10)

## Shuffle the data

In [12]:
from sklearn.utils import shuffle
data = shuffle(data)
data.reset_index(inplace = True)
data.head()

,index,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,696,0,1,1.100515,0,0,-0.486337,0,1,0,1
1,636,0,1,0.177063,0,0,-0.488854,0,1,0,1
2,377,0,1,-0.207709,0,2,3.610065,0,0,0,0
3,537,1,0,0.023154,0,0,1.494412,0,0,0,0
4,554,1,0,-0.592481,0,0,-0.491874,0,1,0,1


In [17]:
data = data.drop(['index'], axis = 1)

## Save data in npz file

In [18]:
outputs_all = data['Survived']
inputs_all = data.drop(['Survived'], axis = 1)

In [19]:
train_samples_count = (int)(inputs_all.shape[0] * 0.9)
validation_samples_count = inputs_all.shape[0] - train_samples_count

In [20]:
train_inputs = inputs_all[:train_samples_count]
train_targets = outputs_all[:train_samples_count]

validation_inputs = inputs_all[train_samples_count:]
validation_targets = outputs_all[train_samples_count:]

In [21]:
np.savez('Titanic_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Titanic_data_validation', inputs = validation_inputs, targets = validation_targets)
#np.savez('house_test', inputs = test_inputs, targets = test_outputs)